# Descenso por gradiente completo

En este notebook implementaremos el algoritmo completo de descenso por gradiente. Para validar que funciona al final lo probaremos en el entrenamiento de una red neuronal simple. Usaremos como conjunto de datos que esta incluído en el archivo data.csv. 

@juan1rving



In [1]:
# importamos los paquete necesarios
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# cargamos datos
data = pd.read_csv('data/data.csv', sep=',', engine='python')
data['gre_scaled'] = data['gre'] / 1000
data['gpa_scaled'] = data['gpa'] / 10
data['rank_scaled'] = data['rank'] / 10
X = data[['gre_scaled', 'gpa_scaled']]
y = data['admit']
n_records, n_features = X.shape

# Dividimos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

last_loss = None

# En este ejercicio por propósitos de analizar las salidas utilizaremos la misma semilla para los números aleatorios.
np.random.seed(42)

In [2]:
# Definimos algunas funciones necesarias
# función de activación
def sigmoid(x):
    return 1/(1+np.exp(-x))

# Derivada de f
def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

# función h lineal
def function_h(X, W, b):
    return np.dot(W, X) + b

## Inicialización de los pesos

En un principio no queremos tener todos los pesos en cero porque esto generaría en la salida una predicción nula. Por lo tanto, asignaremos los pesos iniciales de forma aleatoria y cercanos a cero. Otra recomendación es escalar los valores aleatorios es dependencia del número de entradas del nodo (n).

$$w = rand(1,\frac{1}{\sqrt{n}})$$



In [3]:
# Initialize weights. 
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)
bias = 0


In [4]:
# Probemos la precisión de la red antes de entrenarla
test_out = [sigmoid(function_h(x, weights, bias)) > 0.5 for x in X_test.values]
accuracy = accuracy_score(y_test, test_out)
print("Prediction accuracy: {:.3f}".format(accuracy))

# La precisión debe ser mala seguramente.

Prediction accuracy: 0.338


## Hiperparámetros de la red

Los hiperpámetros de la red indican el números de veces que itera el método (épocas-epochs), la taza de aprendizaje (learning rate).


In [5]:
# número de épocas
epochs = 2
# tasa de aprendizaje
learnrate = 0.5

## Descenso por gradiente completo.

El algoritmo de descenso por gradiente de forma iterativa cambia el valor de los pesos de tal forma que se disminuya el error. 

<img src="files/nn_02_simple_nn_dg.png">

En la siguiete celda encontrarás la plantilla del algoritmo. Tu misión, si decides aceptarla, es completar el código faltante para que funcione el entrenamiento.

In [6]:
for e in range(epochs):
    delta_w = np.zeros(weights.shape)
    # Para todos los renglones de ejemplo, asignar a x la entrada, y a y la salida deseada
    for x, y in zip(X_train.values, y_train.values):
        # TODO: calcula la predicción de la red
        # Tip: NumPy ya tiene una función que calcula el producto punto. Recuerda que también los logits tienen que pasar por la función de activación.
        h = function_h(x, weights, bias)
        output = sigmoid(h)

        # TODO: calcula el error
        error = np.multiply((y - output), sigmoid_prime(h))

        # TODO: calcula el incremento
        delta_w += np.multiply(learnrate, np.multiply(error, x))
        
    # TODO: Actualiza los pesos
    weights += delta_w
    
    # Ahora calculemos el error en el conjunto de datos de entrenamiento
    if e % (epochs / 10) == 0:
        out = [sigmoid(function_h(x, weights, bias)) for x in X_train.values]
        loss = np.mean((out - y_train) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

Train loss:  0.30016790491702905



## Evaluemos la exactitud de la red

Recordemos que

$$ Exactitud = \frac{\# aciertos}{\# predicciones} $$


In [7]:
# Cálculo de la precisión
# Cálculo de la precisión (train)
test_out = [sigmoid(function_h(x, weights, bias)) > 0.5 for x in X_train.values]
accuracy = accuracy_score(y_train, test_out)
print("Model train accuracy: {:.3f}".format(accuracy))

# Cálculo de la precisión (test)
test_out = [sigmoid(function_h(x, weights, bias)) > 0.5 for x in X_test.values]
accuracy = accuracy_score(y_test, test_out)
print("Model test accuracy: {:.3f}".format(accuracy))

Model train accuracy: 0.688
Model test accuracy: 0.662


Tal vez obtuvimos la mejor exactidud, pero ¿qué pasará si incrementamos las épocas? O ¿qué es lo que pasará si cambiamos la tasa de aprendizaje?